In [1]:
# Global Variables Set by User **change these values before running script**

# host organization is the ArcGIS Online Org - needs to be admin account
HOST_URL = "https://envisioning.maps.arcgis.com"
HOST_USERNAME = "GBushongENV"

# guest organization is the Enterprise Portal - needs to be admin account
GUEST_URL = "https://pdoscriptdev.esri.com/portal"
GUEST_USERNAME = "admin"

# this group in the host org contains items to share in the collaboration or is empty
HOST_GROUP_ID = "4d7ff4f81d6340428ef290b7de801204"

# this group in the guest org contains items to share in the collaboration or is empty
GUEST_GROUP_ID = "51b8c8ec928e4a1bba1f801395796880"

# this will be the name of the collaboration and the name of the folder created in enterprise
COLLAB_NAME = "Enterprise + Online"

# name of the workspace that the groups will be added to
WORKSPACE_NAME = "Enterprise Workspace"

# description for the collaboration and workspace
DESCRIPTION = "Data sharing sample between Enterprise and Online"

# how many hours from sending the invitation should be valid for (integer)
EXPIRATION = 24

# host contact info
HOST_FIRST = "Joe"
HOST_LAST = "Smith"
HOST_EMAIL = "jsmith@sample.com"

# guest contact info
GUEST_FIRST = "Judy"
GUEST_LAST = "Smith"
GUEST_EMAIL = "jsmith1@sample.com"

# local folder to temporarily save invitation and response docs into
LOCAL_TMP_DIR = "/Users/grac9792/Documents/pdo-scripts"

In [2]:
# import libraries
from arcgis.gis import GIS
from arcgis.gis import Group
from arcgis.gis.admin import Collaboration

In [3]:
# connect to host GIS, the new ArcGIS Online instance
print("Connecting to {} with username {}...".format(HOST_URL, HOST_USERNAME))
host_gis = GIS(HOST_URL, HOST_USERNAME)

# connect to guest GIS, the ArcGIS Enterprise instance
print("Connecting to {} with username {}...".format(GUEST_URL, GUEST_USERNAME))
guest_gis = GIS(GUEST_URL, GUEST_USERNAME, verify_cert=False)

Connecting to https://envisioning.maps.arcgis.com with username GBushongENV...
Enter password: ········
Connecting to https://pdoscriptdev.esri.com/portal with username admin...
Enter password: ········


In [4]:
# search for sharing groups in host gis
host_group = Group(host_gis, HOST_GROUP_ID)
host_group

<Group title:"aec-test-group" owner:GBushongENV>

In [5]:
# create a collaboration
host_collab = host_gis.admin.collaborations.create(name=COLLAB_NAME, description=DESCRIPTION,
                                                  workspace_name=WORKSPACE_NAME,
                                                  workspace_description=DESCRIPTION,
                                                  portal_group_id=host_group.id,
                                                  host_contact_first_name=HOST_FIRST,
                                                  host_contact_last_name=HOST_LAST,
                                                  host_contact_email_address=HOST_EMAIL,
                                                  access_mode='sendAndReceive')

collab_id = host_collab['id']
host_collab

{'id': 'bdccdebf779c4d42b7bee4169b1d324c',
 'name': 'Enterprise + Online',
 'description': 'Data sharing sample between Enterprise and Online',
 'collaborationHostPortalId': '9F2Q47vO7UfnIjjV',
 'config': {'ownerUsername': 'GBushongENV'},
 'created': 1545251772000,
 'modified': 1545251772000}

In [6]:
# set privileges for the new workspace in the new collaboration
config = [{host_collab.workspaces[0]['id']:'sendAndReceive'}]

# invite the guest GIS to the collaboration and download the invitation file
invite_file = host_collab.invite_participant(config_json=config, expiration=EXPIRATION, 
                                             guest_portal_url=GUEST_URL,
                                             save_path=LOCAL_TMP_DIR)

# print the path to the invite file
invite_file

'/Users/grac9792/Documents/pdo-scripts/collaboration-pdoscriptdev.esri.com.invite'

In [7]:
# accept the invitation at the participant GIS
guest_gis.admin.collaborations.accept_invitation(first_name=GUEST_FIRST, last_name=GUEST_LAST,
                                                email=GUEST_EMAIL,
                                                invitation_file=invite_file)

{'success': True}

In [8]:
# get the correct guest collaboration based off shared collaboration id
guest_collab = Collaboration(guest_gis.admin.collaborations, collab_id)
type(guest_collab)

arcgis.gis.admin._collaboration.Collaboration

In [9]:
# now export a response file that can be sent back to the host GIS
response_file = guest_collab.export_invitation(out_folder=LOCAL_TMP_DIR)
response_file

'/Users/grac9792/Documents/pdo-scripts/collaboration-pdoscriptdev.esri.com.response'

In [10]:
# get guest portal id
guest_portal_id = guest_collab.workspaces[0]['participantsWithoutGroupLinks'][0]['portalId']
print(guest_portal_id)

host_portal_id = guest_collab.workspaces[0]['participantGroupLinks'][0]['portalId']
print(host_portal_id)

# find the guest groups to add to collaboration workspace
guest_group = Group(guest_gis, GUEST_GROUP_ID)
guest_group

39dde654-828b-4a97-ad02-92e952a33ff9
9F2Q47vO7UfnIjjV


<Group title:"aec-enterprise-group" owner:admin>

In [12]:
# join a group from the guest GIS to the collaboration
guest_collab.add_group_to_workspace(portal_group=guest_group, workspace=guest_collab.workspaces[0])

# get workspace ID
workspace_id = guest_collab.workspaces[0]['id']

# get portal ID
host_portal_id = guest_collab['collaborationHostPortalId']
host_portal_id

'9F2Q47vO7UfnIjjV'

In [13]:
# accept the collaboration response at the host GIS
host_collab.import_invitation_response(response_file)

{'success': True}

In [13]:
# verify the collaboration
print("Collaborations on host GIS:")
for collab in host_gis.admin.collaborations.list():
    print(collab.name)
    
print("-----------------------------")
print("Collaborations on guest GIS:")
for collab in guest_gis.admin.collaborations.list():
    print(collab.name)

Collaborations on host GIS:
Enterprise + Online
-----------------------------
Collaborations on guest GIS:
Enterprise + Online
